## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('/content/drive/MyDrive/ISM PROJ/Datasets/train.csv')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
###Drop Nan Values
df=df.dropna()


In [6]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [7]:
## Get the Dependent features
y=df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.8.0'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [21]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2571, 4382, 257, 3386, 3202, 2561, 17, 3277, 2836, 829],
 [2474, 2742, 3027, 4113, 1530, 463, 3696],
 [2143, 4086, 855, 3259],
 [4030, 2369, 2243, 4532, 1769, 1397],
 [4442, 1530, 1903, 2401, 2158, 4632, 1530, 3005, 2770, 2846],
 [2505,
  955,
  2261,
  2045,
  4430,
  1245,
  4989,
  978,
  4760,
  3187,
  767,
  3145,
  1010,
  3099,
  3696],
 [498, 3053, 4027, 2998, 1525, 159, 1394, 4804, 2309, 3704, 3290],
 [4950, 1157, 3971, 4977, 3586, 800, 1245, 560, 2309, 3704, 3290],
 [1368, 3251, 2969, 2408, 723, 979, 3785, 2751, 1245, 2813],
 [2762, 1069, 1567, 2355, 3185, 583, 3226, 725],
 [4174, 607, 4759, 4130, 626, 4046, 1592, 4040, 599, 3322, 23],
 [4532, 1005, 3202, 979, 1245, 3586],
 [1580, 3426, 1243, 4433, 2947, 825, 986, 4034, 1750],
 [4062, 16, 4774, 487, 244, 1029, 1885, 2309, 3704, 3290],
 [4098, 1960, 4222, 2347, 4955, 2309, 3704, 3290],
 [1830, 3118, 3415, 455, 1997, 1637, 108, 556, 4443, 4044],
 [4769, 4093, 2742],
 [74, 1057, 3847, 2346, 1245, 1379, 2094, 3696],
 [1685, 20

### Embedding Representation

In [23]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3277 2836  829]
 [   0    0    0 ... 1530  463 3696]
 [   0    0    0 ... 4086  855 3259]
 ...
 [   0    0    0 ... 2309 3704 3290]
 [   0    0    0 ... 3216 4461 1388]
 [   0    0    0 ... 4673 4617 4986]]


In [24]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2571,
       4382,  257, 3386, 3202, 2561,   17, 3277, 2836,  829], dtype=int32)

In [25]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
len(embedded_docs),y.shape

(18285, (18285,))

In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [49]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 25ms/step - loss: 0.3528 - accuracy: 0.8303 - val_loss: 0.1952 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1455 - accuracy: 0.9419 - val_loss: 0.1942 - val_accuracy: 0.9180
Epoch 3/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1068 - accuracy: 0.9626 - val_loss: 0.2087 - val_accuracy: 0.9228
Epoch 4/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0785 - accuracy: 0.9713 - val_loss: 0.2272 - val_accuracy: 0.9163
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0604 - accuracy: 0.9797 - val_loss: 0.3007 - val_accuracy: 0.9157
Epoch 6/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0407 - accuracy: 0.9850 - val_loss: 0.3305 - val_accuracy: 0.9092
Epoch 7/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0298 - accuracy: 0.9903 - val_loss: 0.3890 - val_accuracy: 0.9150

### Adding Dropout 

In [48]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [55]:
y_pred = model.predict(X_test)
y_pred[1]

array([0.00060427], dtype=float32)

In [56]:
class_x=np.argmax(y_pred,axis=1)
class_x

array([0, 0, 0, ..., 0, 0, 0])

In [57]:
from sklearn.metrics import confusion_matrix

In [58]:
confusion_matrix(y_test,class_x)

array([[3419,    0],
       [2616,    0]])

In [59]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,class_x)

0.5665285832642917